**Instalação das bibliotecas que serão utilizadas** - Este bloco não precisa mais executar.

In [ ]:
pip install pandas_datareader
pip install numpy
pip install yfinance
pip install pandas
pip install scikit-learn
pip install nltk

In [ ]:
## Inicío do código
!mv acoes-listadas-b3 acoes-listadas-b3.csv

In [ ]:
import pandas as pd
import yfinance as yf

def obter_dados_acao(symbol):
    try:
        acao = yf.Ticker(symbol)
        dados = acao.info
        return dados
    except Exception as e:
        print(f"Erro ao obter dados da ação {symbol}: {str(e)}")
        return None

def obter_dados_historicos(ticker, start_date, end_date):
    # Obter dados históricos de ações usando a biblioteca yfinance
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    return stock_data

def calcular_medias_moveis(data, short_window, long_window):
    # Calcular médias móveis simples
    data['Short_MA'] = data['Close'].rolling(window=short_window, min_periods=1).mean()
    data['Long_MA'] = data['Close'].rolling(window=long_window, min_periods=1).mean()

def gerar_recomendacao(data):
    # Gerar recomendação com base nas médias móveis
    if data['Short_MA'].iloc[-1] > data['Long_MA'].iloc[-1]:
        recomendacao = 'Compra'
    elif data['Short_MA'].iloc[-1] < data['Long_MA'].iloc[-1]:
        recomendacao = 'Venda'
    else:
        recomendacao = 'Manter'

    # Calcular porcentagem de variação
    porcentagem_variacao = ((data['Short_MA'].iloc[-1] - data['Long_MA'].iloc[-1]) / data['Long_MA'].iloc[-1]) * 100

    return recomendacao, porcentagem_variacao

def analisar_acao_fundamental(symbol):
    # Obter dados da ação
    dados_acao = obter_dados_acao(symbol)

    if dados_acao is not None:
        dividend_yield = dados_acao.get('dividendYield', 0)
        margem_liquida = dados_acao.get('profitMargins', 0)
        preco_lucro = dados_acao.get('trailingPE', 0)
        preco_valor_patrimonial = dados_acao.get('priceToBook', 0)

        crit_eh_valido = True

        if dividend_yield <= 0.04 - 0.01:
            crit_eh_valido = False
            print(f"Ação {dados_acao['symbol']} não atende ao critério de dividend yield.")

        if margem_liquida < 0.1 - 0.01:
            crit_eh_valido = False
            print(f"Ação {dados_acao['symbol']} não atende ao critério de margem líquida.")

        if preco_lucro >= 10:
            crit_eh_valido = False
            print(f"Ação {dados_acao['symbol']} não atende ao critério de P/L.")

        if preco_valor_patrimonial >= 1.3:
            crit_eh_valido = False
            print(f"Ação {dados_acao['symbol']} não atende ao critério de P/VP.")

        if crit_eh_valido:
            print(f"Recomendação fundamental: Compra da ação {dados_acao['symbol']}")
    else:
        print("Dados da ação não disponíveis.")

def analisar_acao_tecnica(symbol, start_date, end_date, short_window, long_window):
    # Obter dados históricos
    stock_data = obter_dados_historicos(symbol, start_date, end_date)

    if stock_data is not None:
        # Calcular médias móveis
        calcular_medias_moveis(stock_data, short_window, long_window)

        # Gerar recomendação
        recomendacao, porcentagem_variacao = gerar_recomendacao(stock_data)

        # Exibir resultados
        print(f"Recomendação técnica para {symbol}: {recomendacao}")
        print(f"Estimativa de variação: {porcentagem_variacao:.2f}%")
    else:
        print("Dados históricos da ação não disponíveis.")

def analisar_acoes(start_date, end_date, short_window, long_window, lista_acoes):
    for symbol in lista_acoes:
        print(f"\nAnalisando ação: {symbol}")
        analisar_acao_fundamental(symbol)
        analisar_acao_tecnica(symbol, start_date, end_date, short_window, long_window)

if __name__ == "__main__":
    # Configurar parâmetros
    start_date = '2023-01-12'
    end_date = '2024-01-12'
    short_window = 50
    long_window = 200

    # Lista de símbolos de ações
    lista_acoes = ['MTRE3.SA', 'JHSF3.SA']  # Adicione mais ações conforme necessário

    # Analisar ações
    analisar_acoes(start_date, end_date, short_window, long_window, lista_acoes)


/usr/local/lib/python3.10/dist-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()



Analisando ação: MTRE3.SA


[*********************100%%**********************]  1 of 1 completed

Recomendação fundamental: Compra da ação MTRE3.SA
Recomendação técnica para MTRE3.SA: Venda
Estimativa de variação: -5.07%



Analisando ação: JHSF3.SA
Recomendação fundamental: Compra da ação JHSF3.SA


[*********************100%%**********************]  1 of 1 completed

Recomendação técnica para JHSF3.SA: Compra
Estimativa de variação: 1.21%
